## 4. Register CSV Data With Athena

Code learned and followed from AAI 540 Lab 2

In [2]:
!pip install -q PyAthena


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import boto3
import sagemaker
import pandas as pd
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
df = pd.read_csv('s3://sagemaker-us-east-1-911199926915/Smart_Grid/data/processeddata/pdata.csv')

In [4]:
df

,timestamp,kWh,weekday,hour,temperature,Holiday,year,month
0,2011-11-23 09:00:00,0.284500,2,9.0,4.84,-1.0,2011,11
1,2011-11-23 10:00:00,0.153333,2,10.0,5.74,-1.0,2011,11
2,2011-11-23 11:00:00,0.110286,2,11.0,7.67,-1.0,2011,11
3,2011-11-23 12:00:00,0.394222,2,12.0,8.84,-1.0,2011,11
4,2011-11-23 13:00:00,0.258333,2,13.0,9.87,-1.0,2011,11
...,...,...,...,...,...,...,...,...
19857,2014-02-27 20:00:00,0.347004,3,20.0,5.94,-1.0,2014,2
19858,2014-02-27 21:00:00,0.334706,3,21.0,5.03,-1.0,2014,2
19859,2014-02-27 22:00:00,0.299760,3,22.0,4.10,-1.0,2014,2
19860,2014-02-27 23:00:00,0.252274,3,23.0,3.93,-1.0,2014,2


In [5]:
from pyathena import connect

In [6]:
s3_private_path_meterdatacsv = "s3://sagemaker-us-east-1-911199926915/Smart_Grid/data/processeddata/"

In [7]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [8]:
# Set Athena parameters
database_name = "dsoawsfp"
table_name_csv = "meterdata_csv"

In [9]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [10]:
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         timestamp timestamp,
         kWh float,
         weekday int,
         hour float,
         temperature float,
         Holiday float,
         year int,
         month int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_private_path_meterdatacsv
)
# Print SQL Query
print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS dsoawsfp.meterdata_csv(
         timestamp timestamp,
         kWh float,
         weekday int,
         hour float,
         temperature float,
         Holiday float,
         year int,
         month int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-911199926915/Smart_Grid/data/processeddata/'
TBLPROPERTIES ('skip.header.line.count'='1')


In [11]:

pd.read_sql(statement, conn)

/tmp/ipykernel_309/928649848.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [12]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_309/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,meterdata_csv


In [13]:
statement = """SELECT * FROM {}.{} LIMIT 960""".format(
    database_name, table_name_csv
)

print(statement)

SELECT * FROM dsoawsfp.meterdata_csv LIMIT 960


In [14]:
df = pd.read_sql(statement, conn)
df

/tmp/ipykernel_309/8222894.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,timestamp,kwh,weekday,hour,temperature,holiday,year,month
0,2011-11-23 09:00:00,0.284500,2,9.0,4.84,-1.0,2011,11
1,2011-11-23 10:00:00,0.153333,2,10.0,5.74,-1.0,2011,11
2,2011-11-23 11:00:00,0.110286,2,11.0,7.67,-1.0,2011,11
3,2011-11-23 12:00:00,0.394222,2,12.0,8.84,-1.0,2011,11
4,2011-11-23 13:00:00,0.258333,2,13.0,9.87,-1.0,2011,11
...,...,...,...,...,...,...,...,...
955,2012-01-02 04:00:00,0.138929,0,4.0,5.77,-1.0,2012,1
956,2012-01-02 05:00:00,0.141581,0,5.0,5.71,-1.0,2012,1
957,2012-01-02 06:00:00,0.148582,0,6.0,5.27,-1.0,2012,1
958,2012-01-02 07:00:00,0.173377,0,7.0,5.26,-1.0,2012,1


In [15]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>